# Подготовка данных

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import pandas as pd

df = pd.read_csv('/content/out.csv')
print('length of data: ', len(df))
df = df.reindex(columns=['response', 'context_1', 'context_2', 'context_3'])
df = df.iloc[4:, :]
df = df.dropna()
df.isnull().sum()

length of data:  254


response     0
context_1    0
context_2    0
context_3    0
dtype: int64

In [4]:
df.iloc[:]

,response,context_1,context_2,context_3
4,Окей\nНет пока что,сиди кукуй \\nв лагере\nпока что\nвремя будет ...,Да,ч происходит\nахахахх
5,тебе чеченцы должны были позвонить,Окей\nНет пока что,сиди кукуй \\nв лагере\nпока что\nвремя будет ...,Да
6,Хорошо,тебе чеченцы должны были позвонить,Окей\nНет пока что,сиди кукуй \\nв лагере\nпока что\nвремя будет ...
7,короче позвонят тебе\nназначишь там время\nког...,Хорошо,тебе чеченцы должны были позвонить,Окей\nНет пока что
8,"Хорошо, без проблем",короче позвонят тебе\nназначишь там время\nког...,Хорошо,тебе чеченцы должны были позвонить
...,...,...,...,...
249,Просто я сейчас хотел примерно сориентироватьс...,по расписанию,Я просто не понимаю что будет на скринах\nТипа...,можно будет во время самого хакатона наврен
250,Можно,Просто я сейчас хотел примерно сориентироватьс...,по расписанию,Я просто не понимаю что будет на скринах\nТипа...
251,"О\nНапиши плез, тип что конретно будет на скри...",Можно,Просто я сейчас хотел примерно сориентироватьс...,по расписанию
252,Вот чат для всех\nМожешь вступить узнать,"О\nНапиши плез, тип что конретно будет на скри...",Можно,Просто я сейчас хотел примерно сориентироватьс...


In [5]:
def clear_data_1(x):
  if x != None and x[-1] not in ['.', '?', '!']:
    return x + '.'
  else:
    return x
def clear_data_2(x):
    if x != None:
      return x.replace('\n', ' ') .replace('\\', '')
    else:
      return x

In [6]:
for col in df.columns:
  df[col] = df[col].map(clear_data_1)
  df[col] = df[col].map(clear_data_2)

In [7]:
df.iloc[1]

response                   тебе чеченцы должны были позвонить.
context_1                                   Окей Нет пока что.
context_2    сиди кукуй  в лагере пока что время будет у те...
context_3                                                  Да.
Name: 5, dtype: object

# Обучение модели
## Решение основано на информации из видео https://www.youtube.com/watch?v=eereDVZeE7Y

In [8]:
from sklearn.model_selection import train_test_split
trn_df, val_df = train_test_split(df, test_size=0.1)
print('Length of Training Data: ', len(trn_df))
print('Length of Val Data: ', len(val_df))

Length of Training Data:  171
Length of Val Data:  20


In [9]:
import os
import pickle
import logging
from transformers import PreTrainedTokenizer, MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pathlib import Path
from typing import Dict, List, Tuple

logger = logging.getLogger(__name__)

def construct_convo(row, tokenizer):
  print(row)
  print("---")
  print(tokenizer.encode(row[0]))
  print("---")
  print(tokenizer.encode(row[-1]))
  print("---")
  conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
  print(conv)
  flatten = lambda l: [item for sublist in l for item in sublist]
  conv = flatten(conv)
  print(conv)
  return conv

class ConversationDataset(Dataset):
  def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):
    block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

    directory = args.cache_dir
    cached_features_file = os.path.join(
        directory, args.model_type + "_cached_lm_" + str(block_size)
    )

    if os.path.exists(cached_features_file) and not args.overwrite_cache:
      logger.info("Loading features from cached file %s", cached_features_file)
      with open(cached_features_file, "rb") as handle:
          self.examples = pickle.load(handle)
    else:
      logger.info("Creating features from dataset file at %s", directory)

      self.examples = []
      for _, row in df.iterrows():
          conv = construct_convo(row, tokenizer)
          self.examples.append(conv)

      logger.info("Saving features into cached file %s", cached_features_file)
      with open(cached_features_file, "wb") as handle:
        pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)

In [10]:
for _, row in trn_df.iterrows():
  conv = construct_convo(row, tokenizer)
  break


response                     богдан ты где.
context_1                      ща я создам.
context_2           ч по зуму ссылка будет?
context_3    Вот да Можно туда попробовать.
Name: 186, dtype: object
---
[296, 556, 268, 694, 988, 18]
---
[7494, 581, 4950, 3336, 11543, 18]
---
[[7494, 581, 4950, 3336, 11543, 18, 50261], [283, 334, 325, 38953, 16943, 840, 35, 50261], [735, 417, 1650, 277, 18, 50261], [296, 556, 268, 694, 988, 18, 50261]]
[7494, 581, 4950, 3336, 11543, 18, 50261, 283, 334, 325, 38953, 16943, 840, 35, 50261, 735, 417, 1650, 277, 18, 50261, 296, 556, 268, 694, 988, 18, 50261]


In [11]:
# Caching and storing of data/checkpoints
import re
import random
import glob
import shutil
import numpy as np

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
   return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

def set_seed(args):
   random.seed(args.seed)
   np.random.seed(args.seed)
   torch.manual_seed(args.seed)
   if args.n_gpu > 0:
       torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
   ordering_and_checkpoint_path = []


   glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))


   for path in glob_checkpoints:
       if use_mtime:
           ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
       else:
           regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
           if regex_match and regex_match.groups():
               ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))


   checkpoints_sorted = sorted(ordering_and_checkpoint_path)
   checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
   return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
   if not args.save_total_limit:
       return
   if args.save_total_limit <= 0:
       return


   # Check if we should delete older checkpoint(s)
   checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
   if len(checkpoints_sorted) <= args.save_total_limit:
       return

   number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
   checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
   for checkpoint in checkpoints_to_be_deleted:
       logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
       shutil.rmtree(checkpoint)

In [12]:
from transformers import PreTrainedModel, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm, trange

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )
    print("Batch size: ", args.train_batch_size)
    print("Data Loader length: ", len(train_dataloader))

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                print("Batch shape: ", batch.shape)
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [13]:
# Main runner
from transformers import AutoConfig

def main(df_trn, df_val):
    args = Args()

    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir='/content/'+args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir='/content/'+args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir='/content/'+args.cache_dir,
    )
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)
        print(train_dataset)
        print("Train Dataset Length: ", len(train_dataset))
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [14]:
class Args():
    def __init__(self):
        self.output_dir = 'rudialoGPT-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'tinkoff-ai/ruDialoGPT-medium'
        self.config_name = 'tinkoff-ai/ruDialoGPT-medium'
        self.tokenizer_name = 'tinkoff-ai/ruDialoGPT-medium'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'


args = Args()

In [15]:
main(trn_df, val_df)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


response                     богдан ты где.
context_1                      ща я создам.
context_2           ч по зуму ссылка будет?
context_3    Вот да Можно туда попробовать.
Name: 186, dtype: object
---
[296, 556, 268, 694, 988, 18]
---
[7494, 581, 4950, 3336, 11543, 18]
---
[[7494, 581, 4950, 3336, 11543, 18, 50261], [283, 334, 325, 38953, 16943, 840, 35, 50261], [735, 417, 1650, 277, 18, 50261], [296, 556, 268, 694, 988, 18, 50261]]
[7494, 581, 4950, 3336, 11543, 18, 50261, 283, 334, 325, 38953, 16943, 840, 35, 50261, 735, 417, 1650, 277, 18, 50261, 296, 556, 268, 694, 988, 18, 50261]
response     можно попробовать так поиграться, если норм.. ...
context_1    оки Лого вставил в презу вроде заебись ща еще ...
context_2                         Давай тогда в 21:30 засядем.
context_3    но я могу на 15 минут зайти потом в 21.30 осво...
Name: 157, dtype: object
---
[2605, 11543, 453, 2867, 844, 506, 16, 827, 3343, 498, 4528, 424, 7285, 3047, 6529, 289, 1122, 334, 5536, 468, 3136, 428, 6

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50262. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc



[983, 379, 602, 29977, 17966, 1398, 459, 3240, 17966, 2837, 1397, 656, 282, 7823, 5087, 2419, 322, 6429, 649, 439, 1472, 12779, 29950, 457, 18, 263, 18, 14917, 5087, 6908, 4778, 40727, 18]
---
[16745, 912, 6645, 27895, 2789, 282, 481, 289, 1650, 339, 1143, 12094, 365, 1750, 1660, 2419, 1038, 43667, 867, 670, 17672, 3504, 304, 942, 935, 322, 334, 430, 367, 4966, 41914, 16520, 289, 7940, 5690, 10434, 334, 311, 406, 326, 554, 411, 2759, 4966, 18]
---
[[16745, 912, 6645, 27895, 2789, 282, 481, 289, 1650, 339, 1143, 12094, 365, 1750, 1660, 2419, 1038, 43667, 867, 670, 17672, 3504, 304, 942, 935, 322, 334, 430, 367, 4966, 41914, 16520, 289, 7940, 5690, 10434, 334, 311, 406, 326, 554, 411, 2759, 4966, 18, 50261], [6161, 656, 1139, 3984, 697, 39917, 889, 840, 322, 453, 11726, 315, 832, 313, 538, 827, 656, 697, 3984, 4823, 294, 1190, 459, 14895, 4538, 3984, 10919, 13064, 360, 1810, 5257, 374, 282, 813, 26605, 15127, 8010, 541, 9917, 354, 4320, 2459, 374, 656, 45205, 318, 459, 1299, 225, 1513, 

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Iteration:   0%|          | 0/42 [00:00<?, ?it/s]

Iteration:   0%|          | 0/42 [00:00<?, ?it/s]

response     Компании будут платить Либо можно также платит...
context_1                         А за счет чего доход будет ?
context_2    Мне небольшая идея пришла Я ее написал в текст...
context_3    Там в текстовый документ напишите Ч добавили б...
Name: 65, dtype: object
---
[7530, 7421, 1747, 10691, 27350, 942, 1038, 10691, 15329, 7309, 289, 360, 3336, 4823, 13064, 18]
---
[15995, 282, 6529, 1742, 8501, 39478, 688, 34259, 373, 670, 688, 282, 335, 942, 2048, 18]
---
[[15995, 282, 6529, 1742, 8501, 39478, 688, 34259, 373, 670, 688, 282, 335, 942, 2048, 18, 50261], [11233, 15929, 7865, 6712, 629, 722, 6463, 282, 6529, 1742, 22841, 309, 955, 2557, 3782, 10492, 306, 3666, 985, 7865, 30, 365, 3778, 17, 35368, 16, 1029, 30552, 39524, 17978, 15329, 1356, 289, 36760, 5754, 18, 2424, 22642, 30, 430, 18, 4406, 2491, 1733, 16, 670, 1139, 17978, 15329, 1262, 16, 961, 5024, 2083, 6535, 282, 9524, 16, 282, 2114, 322, 1698, 3089, 428, 965, 21001, 18, 23976, 16, 1379, 14002, 10928, 6721, 773, 3089

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'perplexity_': tensor(2.3302)}

In [16]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelForCausalLM.from_pretrained('/content/rudialoGPT-small')

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
        bot_input_ids,
        top_k=10,
        top_p=0.95,
        num_beams=3,
        num_return_sequences=1,
        do_sample=True,
        no_repeat_ngram_size=2,
        temperature=1.2,
        repetition_penalty=1.2,
        length_penalty=1.0,
        eos_token_id=50257,
        max_new_tokens=20)
    print(chat_history_ids)
    # pretty print last ouput tokens from bot
    print("HubermanBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


>> User:Привет


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


tensor([[37954, 50261]])
tensor([[37954, 50261,   618,   890, 16847,   890,   890,    18, 50261,  4529,
            16,   417,  1320,   381,   803,  3684, 15520,   282,  1533,    18,
          3161,  1056]])
HubermanBot: Ахахаххаха.Да, я тоже об этом думаю Завтра в 10.30 соз


KeyboardInterrupt: ignored

# Публикация для дальнейшего использования в телеграм боте (API)

In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
model.push_to_hub("MaximMS/MySecondModel", create_pr=1)

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MaximMS/MySecondModel/commit/3b296368f35abd57ec3afe0edcbf0def4bfcfa4f', commit_message='Upload model', commit_description='', oid='3b296368f35abd57ec3afe0edcbf0def4bfcfa4f', pr_url='https://huggingface.co/MaximMS/MySecondModel/discussions/1', pr_revision='refs/pr/1', pr_num=1)

In [19]:
tokenizer.push_to_hub("MaximMS/MySecondModel", create_pr=1)

CommitInfo(commit_url='https://huggingface.co/MaximMS/MySecondModel/commit/2f4789103a88e4dfb9a043be20ec857ae5747dae', commit_message='Upload tokenizer', commit_description='', oid='2f4789103a88e4dfb9a043be20ec857ae5747dae', pr_url='https://huggingface.co/MaximMS/MySecondModel/discussions/2', pr_revision='refs/pr/2', pr_num=2)